In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 132684 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
X_train = pd.read_pickle('drive/train_max_x')
#X_train = pd.read_pickle('file/test_max_x')

In [0]:
X_train = X_train.astype('float32')
X_train = X_train/255
X_train_copy = X_train.copy()

In [5]:
import keras
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout, AveragePooling2D
from keras.datasets import mnist
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import cv2 as cv

X = pd.read_pickle('drive/train_max_x')
y = pd.read_csv('drive/train_max_y.csv')
y = y.drop(columns='Id')        #drop id
y = y.to_numpy()                #convert y to ndarray

test = pd.read_pickle('drive/test_max_x')
test = test.reshape(10000, 128, 128, 1)

X_train = X[0:50000].reshape(50000, 128, 128, 1)
y_train = y[0:50000]

y_train = to_categorical(y_train)

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(128, 128, 1)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=5, activation='relu', input_shape=(128, 128, 1)))
model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(256, kernel_size=3, activation='relu', input_shape=(128, 128, 1)))
model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 126, 126, 64)      640       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 124, 124, 64)      36928     
_________________________________________________________________
batch_normalization_8 (Batch (None, 124, 124, 64)      256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 58, 58, 128)       204928    
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 54, 54, 128)      

In [0]:
from keras.callbacks import EarlyStopping
from keras import optimizers

nadam = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00005)

model.fit(X_train, y_train, validation_split=0.04, epochs=20, callbacks=[early_stopping], batch_size=32)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 48000 samples, validate on 2000 samples
Epoch 1/20
48000/48000 [==============================] - 108s 2ms/step - loss: 1.9118 - acc: 0.2580 - val_loss: 1.8718 - val_acc: 0.2675
Epoch 2/20
48000/48000 [==============================] - 98s 2ms/step - loss: 1.7033 - acc: 0.3447 - val_loss: 1.5073 - val_acc: 0.4235
Epoch 3/20
48000/48000 [==============================] - 97s 2ms/step - loss: 1.3131 - acc: 0.5220 - val_loss: 1.2247 - val_acc: 0.6075
Epoch 4/20
48000/48000 [==============================] - 97s 2ms/step - loss: 0.7880 - acc: 0.7387 - val_loss: 0.5903 - val_acc: 0.8080
Epoch 5/20
48000/48000 [==============================] - 97s 2ms/step - loss: 0.5453 - acc: 0.8340 - val_loss: 0.3971 - val_acc: 0.8740
Epoch 6/20
48000/48000 [==============================] - 97s 2ms/step - loss: 0.4188 - acc: 0.8763 - val_loss: 0.2803 - val_acc: 0.9125
Epoch 7/20
48000/48000 [======

In [0]:
nadam = keras.optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.1, epochs=20, callbacks=[early_stopping], batch_size=16)

Train on 46000 samples, validate on 4000 samples
Epoch 1/20
46000/46000 [==============================] - 274s 6ms/step - loss: 0.1574 - acc: 0.9558 - val_loss: 0.0775 - val_acc: 0.9770
Epoch 2/20
46000/46000 [==============================] - 271s 6ms/step - loss: 0.1412 - acc: 0.9607 - val_loss: 0.0737 - val_acc: 0.9800
Epoch 3/20
46000/46000 [==============================] - 269s 6ms/step - loss: 0.1277 - acc: 0.9640 - val_loss: 0.0712 - val_acc: 0.9792
Epoch 4/20
46000/46000 [==============================] - 268s 6ms/step - loss: 0.1251 - acc: 0.9645 - val_loss: 0.0656 - val_acc: 0.9810
Epoch 5/20
46000/46000 [==============================] - 268s 6ms/step - loss: 0.1161 - acc: 0.9675 - val_loss: 0.0671 - val_acc: 0.9810
Epoch 6/20
46000/46000 [==============================] - 268s 6ms/step - loss: 0.1122 - acc: 0.9677 - val_loss: 0.0674 - val_acc: 0.9820
Epoch 7/20
46000/46000 [==============================] - 268s 6ms/step - loss: 0.1141 - acc: 0.9674 - val_loss: 0.0653 - v

In [0]:
adam = keras.optimizers.adam(lr=0.0001)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.1, epochs=20, callbacks=[early_stopping], batch_size=16)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 254s 6ms/step - loss: 0.0848 - acc: 0.9751 - val_loss: 0.0561 - val_acc: 0.9854
Epoch 2/20
45000/45000 [==============================] - 231s 5ms/step - loss: 0.0818 - acc: 0.9759 - val_loss: 0.0633 - val_acc: 0.9848
Epoch 3/20
45000/45000 [==============================] - 231s 5ms/step - loss: 0.0795 - acc: 0.9772 - val_loss: 0.0635 - val_acc: 0.9832
Epoch 4/20
45000/45000 [==============================] - 231s 5ms/step - loss: 0.0792 - acc: 0.9767 - val_loss: 0.0613 - val_acc: 0.9846
Epoch 00004: early stopping


In [0]:

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.1, epochs=20, callbacks=[early_stopping], batch_size=32)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 211s 5ms/step - loss: 0.0653 - acc: 0.9805 - val_loss: 0.0631 - val_acc: 0.9820
Epoch 2/20
45000/45000 [==============================] - 200s 4ms/step - loss: 0.0590 - acc: 0.9816 - val_loss: 0.0663 - val_acc: 0.9822
Epoch 3/20
45000/45000 [==============================] - 200s 4ms/step - loss: 0.0575 - acc: 0.9819 - val_loss: 0.0622 - val_acc: 0.9822
Epoch 4/20
45000/45000 [==============================] - 199s 4ms/step - loss: 0.0557 - acc: 0.9834 - val_loss: 0.0586 - val_acc: 0.9844
Epoch 5/20
45000/45000 [==============================] - 200s 4ms/step - loss: 0.0545 - acc: 0.9836 - val_loss: 0.0584 - val_acc: 0.9834
Epoch 6/20
45000/45000 [==============================] - 200s 4ms/step - loss: 0.0535 - acc: 0.9833 - val_loss: 0.0573 - val_acc: 0.9848
Epoch 7/20
45000/45000 [==============================] - 200s 4ms/step - loss: 0.0510 - acc: 0.9838 - val_loss: 0.0570 - v

In [0]:
model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[early_stopping], batch_size=16)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 225s 6ms/step - loss: 0.0737 - acc: 0.9795 - val_loss: 0.0473 - val_acc: 0.9875
Epoch 2/20
40000/40000 [==============================] - 223s 6ms/step - loss: 0.0649 - acc: 0.9814 - val_loss: 0.0475 - val_acc: 0.9871
Epoch 3/20
40000/40000 [==============================] - 223s 6ms/step - loss: 0.0673 - acc: 0.9810 - val_loss: 0.0476 - val_acc: 0.9872
Epoch 4/20
40000/40000 [==============================] - 223s 6ms/step - loss: 0.0648 - acc: 0.9814 - val_loss: 0.0430 - val_acc: 0.9888
Epoch 5/20
40000/40000 [==============================] - 223s 6ms/step - loss: 0.0646 - acc: 0.9810 - val_loss: 0.0452 - val_acc: 0.9878
Epoch 6/20
40000/40000 [==============================] - 223s 6ms/step - loss: 0.0612 - acc: 0.9817 - val_loss: 0.0466 - val_acc: 0.9878
Epoch 7/20
40000/40000 [==============================] - 223s 6ms/step - loss: 0.0599 - acc: 0.9826 - val_loss: 0.0489 - 

In [0]:
model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.000001)

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[early_stopping], batch_size=16)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 230s 6ms/step - loss: 0.0734 - acc: 0.9784 - val_loss: 0.0493 - val_acc: 0.9864
Epoch 2/20
40000/40000 [==============================] - 227s 6ms/step - loss: 0.0635 - acc: 0.9807 - val_loss: 0.0516 - val_acc: 0.9864
Epoch 3/20
40000/40000 [==============================] - 228s 6ms/step - loss: 0.0637 - acc: 0.9817 - val_loss: 0.0470 - val_acc: 0.9884
Epoch 4/20
40000/40000 [==============================] - 227s 6ms/step - loss: 0.0613 - acc: 0.9822 - val_loss: 0.0483 - val_acc: 0.9865
Epoch 5/20
40000/40000 [==============================] - 227s 6ms/step - loss: 0.0639 - acc: 0.9809 - val_loss: 0.0464 - val_acc: 0.9876
Epoch 6/20
40000/40000 [==============================] - 227s 6ms/step - loss: 0.0630 - acc: 0.9818 - val_loss: 0.0482 - val_acc: 0.9864
Epoch 7/20
40000/40000 [==============================] - 228s 6ms/step - loss: 0.0582 - acc: 0.9829 - val_loss: 0.0498 - 

In [0]:
adam = keras.optimizers.adam(lr=0.00005)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.000001)

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[early_stopping], batch_size=16)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 217s 5ms/step - loss: 0.0506 - acc: 0.9857 - val_loss: 0.0486 - val_acc: 0.9872
Epoch 2/20
40000/40000 [==============================] - 215s 5ms/step - loss: 0.0513 - acc: 0.9847 - val_loss: 0.0481 - val_acc: 0.9866
Epoch 3/20
40000/40000 [==============================] - 214s 5ms/step - loss: 0.0510 - acc: 0.9852 - val_loss: 0.0477 - val_acc: 0.9873
Epoch 4/20
40000/40000 [==============================] - 214s 5ms/step - loss: 0.0491 - acc: 0.9849 - val_loss: 0.0444 - val_acc: 0.9883
Epoch 5/20
40000/40000 [==============================] - 214s 5ms/step - loss: 0.0495 - acc: 0.9853 - val_loss: 0.0461 - val_acc: 0.9872
Epoch 6/20
40000/40000 [==============================] - 214s 5ms/step - loss: 0.0508 - acc: 0.9848 - val_loss: 0.0471 - val_acc: 0.9874
Epoch 7/20
40000/40000 [==============================] - 214s 5ms/step - loss: 0.0511 - acc: 0.9849 - val_loss: 0.0482 - 

In [0]:
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.0, decay=0.0, nesterov=False)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[early_stopping], batch_size=16)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 200s 5ms/step - loss: 0.0456 - acc: 0.9857 - val_loss: 0.0468 - val_acc: 0.9872
Epoch 2/20
40000/40000 [==============================] - 197s 5ms/step - loss: 0.0468 - acc: 0.9862 - val_loss: 0.0457 - val_acc: 0.9876
Epoch 3/20
40000/40000 [==============================] - 197s 5ms/step - loss: 0.0486 - acc: 0.9849 - val_loss: 0.0458 - val_acc: 0.9876
Epoch 4/20
40000/40000 [==============================] - 197s 5ms/step - loss: 0.0459 - acc: 0.9862 - val_loss: 0.0455 - val_acc: 0.9878
Epoch 5/20
40000/40000 [==============================] - 197s 5ms/step - loss: 0.0420 - acc: 0.9870 - val_loss: 0.0452 - val_acc: 0.9878
Epoch 6/20
40000/40000 [==============================] - 196s 5ms/step - loss: 0.0472 - acc: 0.9859 - val_loss: 0.0448 - val_acc: 0.9879
Epoch 7/20
40000/40000 [==============================] - 197s 5ms/step - loss: 0.0474 - acc: 0.9855 - val_loss: 0.0453 - 

In [0]:
sgd = keras.optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[early_stopping], batch_size=8)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
14696/40000 [==========>...................] - ETA: 2:52 - loss: 0.7195 - acc: 0.8415

KeyboardInterrupt: ignored

In [0]:
sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.0, decay=0.0, nesterov=False)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto', min_delta=0.00001)

model.fit(X_train, y_train, validation_split=0.2, epochs=20, callbacks=[early_stopping], batch_size=8)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 278s 7ms/step - loss: 0.1200 - acc: 0.9742 - val_loss: 0.0473 - val_acc: 0.9874
Epoch 2/20
40000/40000 [==============================] - 278s 7ms/step - loss: 0.0899 - acc: 0.9793 - val_loss: 0.0453 - val_acc: 0.9880
Epoch 3/20
40000/40000 [==============================] - 276s 7ms/step - loss: 0.0997 - acc: 0.9781 - val_loss: 0.0478 - val_acc: 0.9878
Epoch 4/20
40000/40000 [==============================] - 274s 7ms/step - loss: 0.1020 - acc: 0.9777 - val_loss: 0.0465 - val_acc: 0.9877
Epoch 5/20
40000/40000 [==============================] - 274s 7ms/step - loss: 0.0945 - acc: 0.9783 - val_loss: 0.0459 - val_acc: 0.9882
Epoch 00005: early stopping


In [0]:

result = model.predict_classes(test)

In [0]:
len(result)

10000

In [0]:
from google.colab import files
np.savetxt('proj3_pred20.csv', np.dstack((np.arange(0, result.size),result))[0],"%s,%s",header="Id,Label")
files.download('proj3_pred20.csv')

In [0]:
from sklearn.cluster import KMeans

X_train_image = []
for i in range(len(X_train)):
    X_image = X_train[i].copy()
    X_image_copy = X_train_copy[i].copy()
    coordinates = []
    coordinates_add = []

    for x in range(128):
        for y in range(128):
            if X_image[x][y] == 1:
                coordinates.append([x,y])
            else:
                coordinates_add.append([x,y])

                
                
    kmeans = KMeans(n_clusters=3)
    para = kmeans.fit(coordinates)
    y_kmeans = para.predict(coordinates)
    
    number_one = []
    number_two = []
    number_three = []
    x_min = 128
    y_min = 128
    x_max = 0
    y_max = 0
    row = []
    col = []

    for label in range(len(y_kmeans)):
        if y_kmeans[label] == 0.:
            number_one.append(coordinates[label])
        elif y_kmeans[label] == 1.:
            number_two.append(coordinates[label])
        else:
            number_three.append(coordinates[label])
    
    
    for image in [number_one, number_two, number_three]:
        for i in range(len(image)):
            if image[i][0] < x_min:
                x_min = image[i][0]
            if image[i][1] < y_min:
                y_min = image[i][1]
            if image[i][0] > x_max:
                x_max = image[i][0]
            if image[i][1] > y_max:
                y_max = image[i][1]
        for i in range(len(coordinates_add)) :
            if x_max >= coordinates_add[i][0] >= x_min and y_max >= coordinates_add[i][1] >= y_min:
                image.append(coordinates_add[i])
        x_min = 128
        y_min = 128
        x_max = 0
        y_max = 0 

        
    new_image = np.zeros((128,128))            
    for old_image in [number_one, number_two, number_three]:
        for i in range(128):
            for j in range(128):
                if [i,j] in old_image:
                    new_image[i][j] = X_image_copy[i][j]
    #plt.imshow(new_image)
    #plt.show()
    X_train_image.append(new_image)

In [0]:
import pickle
X_train_image = np.array(X_train_image)
file = open('modified_X_train_data.pickle', 'wb')
pickle.dump(X_train_image, file)
file.close()


NameError: ignored

In [0]:
from google.colab import files
files.download(modified_X_train_data.pickle’)
